In [3]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import geocoder # import geocoder



Load the webpage on python and parse html

In [20]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_coord = 'https://cocl.us/Geospatial_data'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find_all('table')[0]

for the first table we have, which is the postal code table, load all the entry to a dataframe

In [5]:
# let's parse and add to the dataframe row by row
# make a temp table which has one row and collect data on this row.
tabletemp=[]
for row in table.find_all('tr'):
    rowtemp=[]
    for td in row.find_all('td'):
        rowtemp = rowtemp + [td.get_text()]
    tabletemp.append(rowtemp)
    headtemp=[]
for th in table.find_all('th'):
    headtemp= headtemp + [th.get_text()]
tabletemp[0]= headtemp

Get rid of all "\n" in the table

In [6]:
for eachrow in tabletemp:
    for i, word in enumerate(eachrow):
        eachrow[i] = word.strip('\n')

In [31]:
new_table = pd.DataFrame(tabletemp[1:],columns=tabletemp[0])

In [32]:
delete_index = new_table[new_table['Borough']=='Not assigned'].index
new_table.drop(delete_index, axis=0, inplace=True)

In [33]:
new_table.reset_index(drop=True, inplace=True)

In [34]:
neigh_na_index = new_table[new_table['Neighbourhood']=='Not assigned'].index
new_table.iloc[neigh_na_index, 2] = new_table[new_table['Neighbourhood']=='Not assigned']['Borough']

In [35]:
new_table=new_table.groupby(['Postcode','Borough'], as_index=False)['Neighbourhood'].agg(lambda nbh: ', '.join(nbh))

In [36]:
new_table.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [37]:
new_table['Postcode'][0]

'M1B'

In [38]:
c=pd.read_csv(url_coord)


In [39]:
new_table = new_table.join(c[['Latitude','Longitude']])

In [40]:
new_table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
